# Szyfrowanie w pełni holomorficzne - Laboratorium 01

## Pierścienie ilorazowe wielomianów

Obiektem matematycznym powiązanym z ciałami Galois i używanym w kryptografi homomorficznej jest pierścień ilorazowy wielomianów $\mathbb{Z}_p[X]/W(X)$, gdzie $W(X)$ jest danym, konkretnym wielomianem stopnia $n$ a $p$ konkretną liczbą (najczęściej pierwszą).

$\mathbb{Z}_p[X]$ oznacza tutaj pierścień wielomianów dowolnych stopni o współczynnikach będących liczbami z $\mathbb{Z}_p$. Żeby otrzymać reprezentację wielomianu z $\mathbb{Z}[X]$ (tzn. wielomianu o współczynnikach całkowitych) w $\mathbb{Z}_p[X]$ należy obliczyć reprezentację jego współczynników $\mod p$.

Pierścień ilorazowy $\mathbb{Z}_p[X]/W(X)$ to mówiąc prostym językiem pierścień reszt z dzielenia wielomianów z $\mathbb{Z}_p[X]$ przez wielomian $W(X)$, czyli reprezentacją danego wielomianu staje się jego reszta z dzielenia przez $W(X)$.

## Zadanie 1.

Zaimplementować w Pythonie pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$ wraz z arytmetyką, tzn. działaniami dodawania (+), odejmowania (-), mnożenia (\*) oraz mnożenia przez `int` (\*).

Dane testowe:

$$w1=7x^6+14x^3$$
$$w2=24x^4-5x^2-7x+13$$
$$w3=23x^5-3x^4+x^3+35x^2+4$$

Reprezentacja w $\mathbb{Z}_{17}[X]/(X^4+1)$:

$$w1=14x^3 + 10x^2$$
$$w2=12x^2 + 10x + 6$$
$$w3=x^3 + x^2 + 11x + 7$$

Arytmetyka:

$$w1+w2=14x^3 + 5x^2 + 10x + 6$$
$$w1*w2=14x^3 + 9x^2 + 2x + 12$$
$$6*w3=6x^3 + 6x^2 + 15x + 8$$
$$w3*6=6x^3 + 6x^2 + 15x + 8$$





In [8]:
import numpy as np
import random

In [9]:
class Ring_4_17:
    def __init__(self):
        self.q = 17
        self.w = np.poly1d([1, 0, 0, 0, 1])
        
    def ring_rep(self, poly):
        return np.poly1d(np.floor(np.polydiv(poly, self.w)[1]) % self.q)
    
    def add_poly(self, poly_1, poly_2):
        return np.poly1d(np.floor(np.polyadd(poly_1, poly_2)) % self.q)

    def sub_poly(self, poly_1, poly_2):
        return np.poly1d(np.floor(np.polysub(poly_1, poly_2)) % self.q)
    
    def mul_poly(self, poly_1, poly_2):
        return np.poly1d(np.floor(np.polydiv(np.polymul(poly_1, poly_2), self.w)[1]) % self.q)

    def mul_poly_int_right(self, poly, num):
        return np.poly1d(np.floor(np.polydiv(np.polymul(poly, np.poly1d([num])), self.w)[1]) % self.q)

    def mul_poly_int_left(self, num, poly):
        return np.poly1d(np.floor(np.polydiv(np.polymul(np.poly1d([num]), poly), self.w)[1]) % self.q)


w1 = np.poly1d([7, 0, 0, 14, 0, 0, 0])
w2 = np.poly1d([24, 0, -5, -7, 13])
w3 = np.poly1d([23, -3, 1, 35, 0, 4]) 

ring = Ring_4_17()

w1 = ring.ring_rep(w1)
w2 = ring.ring_rep(w2)
w3 = ring.ring_rep(w3)

sum_w1_w2 = ring.add_poly(w1, w2)
mul_w1_w2 = ring.mul_poly(w1, w2)
_w3_6 = ring.mul_poly_int_right(w3, 6)
_6_w3 = ring.mul_poly_int_left(6, w3)

print('Reprezentacja w  ℤ17[𝑋]/(𝑋4+1):')
print('w1 = \n', w1)
print('\nw2 = \n', w2)
print('\nw3 = \n', w3)

print('\nArytmetyka:')
print('w1 + w2 = \n', sum_w1_w2)
print('\nw1 * w2 = \n', mul_w1_w2)
print('\n6 * w3 = \n', _6_w3)
print('\nw3 * 6 = \n', _w3_6)

Reprezentacja w  ℤ17[𝑋]/(𝑋4+1):
w1 = 
     3      2
14 x + 10 x

w2 = 
     2
12 x + 10 x + 6

w3 = 
    3     2
1 x + 1 x + 11 x + 7

Arytmetyka:
w1 + w2 = 
     3     2
14 x + 5 x + 10 x + 6

w1 * w2 = 
     3     2
14 x + 9 x + 2 x + 12

6 * w3 = 
    3     2
6 x + 6 x + 15 x + 8

w3 * 6 = 
    3     2
6 x + 6 x + 15 x + 8


## Algorytm BGV (Brakerski, Gentry, Vaikuntanathan 2011)

Parametry kryptosystemu:
- $n$ - stopień wielomianu $X^n+1$
- $q$ - podstawa arytmetyki modularnej
- $t$ - podstawa arytmetyki modularnej plaintextu, $t<<q$
- $\chi$ - dyskretny rokład typu Gaussowskiego
- $R_q=\mathbb{Z}_{q}[X]/(X^n+1)$

W uproszczonym modelu kryptosystemu przyjmijmy $n=4$, $q=17$, $t=2$.

`SecretKeyGen(params) -> sk`

- losujemy wektor $s\in\{-1,0,1\}^n$ z *binomial distribution* (prawdopodobieństwo wylosowania 0 jest największe, a prawdopodobieństwa wylosowania -1 i 1 są sobie równe)
- klucz prywatny $sk=s$
    

`PubKeyGen(sk, params) -> (pk0, pk1)`

- losujemy losowy element $a\in R_q$
- losujemy niewielki (w sensie współczynników) błąd $e\in R_q$ z rozkładu $\chi$
- $pk_0=as+te$
- $pk_1=-a$
- klucz publiczny $pk=(pk_0,pk_1)$

`Encrypt(m, pk, params) -> (c0, c1)`

- losujemy niewielkie (w sensie współczynników) błędy $e_0,e_1\in R_q$ z rozkładu $\chi$
- losujemy wektor $u\in\{-1,0,1\}^n$ z *binomial distribution*
- $c_0=pk_0\cdot u+te_0+m$
- $c_1=pk_1\cdot u+te_1$
- szyfrogram $c=(c_0,c_1)$

`Decrypt(c, sk, params)`

- obliczamy $m=c_0+c_1s\mod q\mod t$
- zwracamy $m$ jako odszyfrowaną wiadomość

## Zadanie 2.

Zaimplementuj uproszczoną wersję algorytmu BGV. Sprawdź poprawność deszyfrowania dla losowo generowanych wiadomości $m$.

In [10]:
class BGV:
    def __init__(self, n, q, t):
        self.n = n
        self.q = q
        self.t = t
        self.ring = Ring_4_17()
    
    def secret_key_gen(self):
        return np.poly1d(np.random.binomial(1, 0.3, self.n - 2))
    
    def pub_key_gen(self, sk, max_coeff = 100):                     
        a = self.ring.ring_rep(np.poly1d(np.random.randint(-max_coeff, max_coeff, self.n)))
        e = self.ring.ring_rep(np.poly1d(np.random.randint(0, 3, self.n - 2)))
        pk0 = self.ring.add_poly(self.ring.mul_poly(a, sk), self.ring.mul_poly_int_left(self.t, e))
        pk1 = self.ring.mul_poly_int_left(-1, a)
        return pk0, pk1
    
    def encrypt(self, m, pk):
        e0 = self.ring.ring_rep(np.poly1d(np.random.randint(0, 3, self.n - 2)))
        e1 = self.ring.ring_rep(np.poly1d(np.random.randint(0, 3, self.n - 2)))
        u = np.poly1d(np.random.binomial(1, 0.3, self.n - 2))
        c0 = self.ring.add_poly(self.ring.add_poly(self.ring.mul_poly(pk[0], u), self.ring.mul_poly_int_left(self.t, e0)), m)
        c1 = self.ring.add_poly(self.ring.mul_poly(pk[1], u), self.ring.mul_poly_int_left(self.t, e1))
        return c0, c1
    
    def decrypt(self, c, sk):
        m = self.ring.add_poly(c[0], self.ring.mul_poly(c[1], sk))
        m = np.poly1d(np.floor(np.floor(m.coef) % self.q) % self.t)
        return m

In [11]:
bgv = BGV(4, 17, 2)
sk = bgv.secret_key_gen()
pk = bgv.pub_key_gen(sk)

iterations = 0
correct = 0

while iterations != 100:
    m = bgv.ring.ring_rep(np.poly1d(np.random.randint(0, 2, 4).tolist()))
    c = bgv.encrypt(m, pk)
    m_dec = bgv.decrypt(c, sk)

    if m_dec == m:
        correct += 1
    
    iterations += 1

print(f'Accuracy: {correct}/{iterations}')

Accuracy: 100/100
